Import statements

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 

In [0]:
# nltk.download('punkt')
# nltk.download('all')
# nltk.download('averaged_perceptron_tagger')

No longer necessary

In [0]:
# data_path='./'
# os.chdir(data_path)
files=glob.glob('*.txt')

sents=[]
tagsP=[]
tagsF=[]
emptyin=[]
for f in files:
    file = open(f)
    df= pd.read_csv(file,delimiter='\t',encoding='utf-8')
    emptyin=df[df['Past'].isnull()].index.tolist()
    st=np.delete(np.array(df['sentence'].values),emptyin)
    ft=np.delete(np.array(df['Future'].values),emptyin)
    pt=np.delete(np.array(df['Past'].values),emptyin)
    sents.extend(list(st))
    tagsF.extend(list(ft))
    tagsP.extend(list(pt))
    print(emptyin)
X=np.asarray(sents)
yf=np.asarray(tagsF)
yp=np.asarray(tagsP)
print(len(X))
print(len(yf))
print(len(yp))

[103, 266, 280, 301, 314, 392, 412, 474, 664, 685, 688, 778, 875, 926, 975]
[49, 82, 195, 258, 533, 541, 577, 648, 726, 942]
[51, 90, 214, 273, 304, 404, 458, 511, 527, 640, 664, 769, 911, 995]
2961
2961
2961


No longer necessary

In [0]:
f='projectivity.txt'
file=open(f)
df= pd.read_csv(file,delimiter='\t',encoding='utf-8')
emptyin=df[df['Past'].isnull()].index
dt=df.loc[:, 'id':'Future']
dt=dt.drop(emptyin)
print(len(dt))
# dt=np.delete(np.array(df.loc[:, 'A':'E'].values),emptyin)


2961


No longer necessary

In [0]:
def split_data_and_store(df):
  train, validation, test = np.split(df, [int(.6*len(df)), int(.8*len(df))])
  train.to_csv('train', sep='\t')
  validation.to_csv('validation', sep='\t')
  test.to_csv('test', sep='\t')
  print('Done!')
  

In [0]:
split_data_and_store(dt)

Compile and run from here.

To load the train and validation data from TSV files.

Arguments: train file, validation file

Returns:

tr_st: Training sentences
tr_ft: Training Future Labels
tr_pt: Trainiing past labels
val_st: Validation sentences
val_ft: Validation Future Labels
val_pt: Validation past labels

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt


Method to load the test data from TSV file.

Arguments: test file

Returns:

ts_st: Test sentences
ts_ft: Test Future Labels
ts_pt: Test past labels

In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

To preprocess sentences:
Removing some special characters, spaces and tabs, and conversion to lower case

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

Converts sentences to lists of features

Uncomment code as needed for features



In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word for (word,pos) in nltk.pos_tag(tokens)]#+' '+pos
#     document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs

This is not necesasry unless the code is being run on a Colab notebook, and the data is stored on Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Path to the location where the data resides

In [0]:
datapath = 'gdrive/My Drive/data/'

Loading the train and validation data

Preprocessing the data

Feature extraction if necessary

In [28]:
#loading train and validation data
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
# tr_st = sent_to_features(tr_st)
# val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [29]:
print(tr_st[1])

friends and adversaries abroad were asking whether america had lost its nerve 


Imports necessary for LSTM 

In [0]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers

# from tensorflow.keras.backend import set_session
import keras.backend as K
import nltk
import string
import numpy as np
import pandas as pd
import re
from tqdm import tqdm


Creating a Keras Tokenizer and fitting it on train and validation data

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

Initializing vocabulary size

In [0]:
vocabulary_size = len(tokenizer.word_index)+1
# print(tokenizer.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_index)
# print(tokenizer.word_docs)


In [35]:
print(vocabulary_size)

6745


Extracting sequences from train and validation data, and padding them to a maximum length

In [36]:

# vocabulary_size = 7000
# tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

sequences = tokenizer.texts_to_sequences(tr_st)
tr_data = pad_sequences(sequences, maxlen=145)
sequences = tokenizer.texts_to_sequences(val_st)
val_data = pad_sequences(sequences, maxlen=145)

print(len(tr_data))
print(len(val_data))

1776
592


Loading Glove embeddings

In [37]:
f = open(datapath+'glove.6B.300d.txt')
embedding_values = {}
for line in tqdm(f):
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype = 'float32')
  embedding_values[word]=coef

400000it [00:26, 15040.24it/s]


Creating an embedding matrix for words 

In [38]:
embedding_matrix = np.zeros((vocabulary_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
  values = embedding_values.get(word)
  if values is not None:
    embedding_matrix[i] = values

100%|██████████| 6744/6744 [00:00<00:00, 301056.72it/s]


In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

BiLSTM for Future task

In [68]:
#Future
K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))

model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

adg= optimizers.Adagrad(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

model.summary()
#model.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=10,batch_size=128) 
model.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=10,batch_size=64)#  validation_data=(val_data,val_ft),

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 300)          2023500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 290)               517360    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 2,559,549
Trainable params: 536,049
Non-trainable params: 2,023,500
_________________________________________________________________
Epoch 1/10
2368/2368 [==============================] - 20s 9ms/step - loss: 0.6417 - acc: 0.6334
Epoch 2/10
2368/2368 [==============================] - 19s 8ms/step - loss: 0.4956 - acc: 0.7720
Epoch 3/10
2368/2368 [==============================] - 21s 9m

Stacked BiLSTM for Future task

In [81]:
#Future LSTM2
K.clear_session()
model2 = Sequential()
model2.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 300)          2023500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 145, 290)          517360    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 290)               505760    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 3,065,309
Trainable params: 1,041,809
Non-trainable params: 2,023,500
_________________________________________________________________


In [82]:
model2.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=10,batch_size=128)  #epoch 15 batch 128 no relu hidden   epoch 7 batch 128 w/ relu hidden

Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 21s 12ms/step - loss: 0.7531 - acc: 0.5225 - val_loss: 0.6418 - val_acc: 0.6166
Epoch 2/10
1776/1776 [==============================] - 17s 9ms/step - loss: 0.6166 - acc: 0.6830 - val_loss: 0.5272 - val_acc: 0.7652
Epoch 3/10
1776/1776 [==============================] - 17s 9ms/step - loss: 0.5449 - acc: 0.7534 - val_loss: 0.5056 - val_acc: 0.7652
Epoch 4/10
1776/1776 [==============================] - 17s 9ms/step - loss: 0.4792 - acc: 0.7889 - val_loss: 0.4991 - val_acc: 0.7838
Epoch 5/10
1776/1776 [==============================] - 17s 10ms/step - loss: 0.4662 - acc: 0.7990 - val_loss: 0.4925 - val_acc: 0.7686
Epoch 6/10
1776/1776 [==============================] - 18s 10ms/step - loss: 0.4202 - acc: 0.8232 - val_loss: 0.4277 - val_acc: 0.8176
Epoch 7/10
1776/1776 [==============================] - 17s 9ms/step - loss: 0.4147 - acc: 0.8153 - val_loss: 0.4505 - val_acc: 0.8041
Epoch

Training on whole training data after parameter tuning, in order to use the model to evaluate the test data

In [0]:
model2.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=10,batch_size=64)

Loading the Test data

preprocessing the data

Extracting sequences

Padding the sequences



In [0]:
testfile=datapath+'test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
#ts_st = sent_to_features(ts_st)
sequences = tokenizer.texts_to_sequences(ts_st)
ts_data = pad_sequences(sequences, maxlen=145)
print(ts_data.shape)
print(ts_ft.shape)
# test_loss, test_acc = model.evaluate(np.append((ts_data,ts_ft),1))
# print('Test Loss: {}'.format(test_loss))
# print('Test Accuracy: {}'.format(test_acc))


# print('Accuaracy = {0:f}'.format(acc))

593
(593, 145)
(593,)
Accuracy: 83.14%


Evaluating the test data for future task 

In [0]:

scores = model.evaluate(ts_data, ts_ft, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))


Generating a classification report for the test data for future task

In [0]:
from sklearn.metrics import classification_report
y_pred = model.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_ft, y_pred))

593/593 [==============================] - 2s 3ms/step
              precision    recall  f1-score   support

         0.0       0.87      0.77      0.82       293
         1.0       0.80      0.89      0.84       300

   micro avg       0.83      0.83      0.83       593
   macro avg       0.84      0.83      0.83       593
weighted avg       0.84      0.83      0.83       593



In [0]:
# scores = model2.evaluate(ts_data, ts_ft, verbose=0)

# print("Accuracy: %.2f%%" % (scores[1]*100))
# y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)

# print(classification_report(ts_ft, y_pred))


BiLSTM model for Past task

In [0]:
#Past

K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 300)          2023500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 290)               517360    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 291       
Total params: 2,541,151
Trainable params: 517,651
Non-trainable params: 2,023,500
_________________________________________________________________


In [0]:
model.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=50,batch_size=128) #eposh 8 batch 128 no relu hidden

Train on 1776 samples, validate on 592 samples
Epoch 1/50
1776/1776 [==============================] - 9s 5ms/step - loss: 0.6458 - acc: 0.6306 - val_loss: 0.6180 - val_acc: 0.6689
Epoch 2/50
1776/1776 [==============================] - 7s 4ms/step - loss: 0.5435 - acc: 0.7523 - val_loss: 0.5476 - val_acc: 0.7584
Epoch 3/50
1776/1776 [==============================] - 7s 4ms/step - loss: 0.4635 - acc: 0.8001 - val_loss: 0.4729 - val_acc: 0.8007
Epoch 4/50
1776/1776 [==============================] - 7s 4ms/step - loss: 0.4320 - acc: 0.8198 - val_loss: 0.5085 - val_acc: 0.7905
Epoch 5/50
1776/1776 [==============================] - 7s 4ms/step - loss: 0.4334 - acc: 0.8091 - val_loss: 0.4906 - val_acc: 0.7872
Epoch 6/50
1776/1776 [==============================] - 7s 4ms/step - loss: 0.3984 - acc: 0.8345 - val_loss: 0.4841 - val_acc: 0.7821
Epoch 7/50
1776/1776 [==============================] - 7s 4ms/step - loss: 0.3907 - acc: 0.8311 - val_loss: 0.4991 - val_acc: 0.7956
Epoch 8/50
1776

Stacked BiLSTM for Past task

In [0]:
#Past LSTM2
K.clear_session()
model2 = Sequential()
model2.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()


In [0]:
model2.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=50,batch_size=128) #eposh 8 batch 128 no relu hidden

Evaluating the test data for future task 

In [0]:

scores = model.evaluate(ts_data, ts_pt, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))


Generating a classification report for the test data for future task

In [0]:
from sklearn.metrics import classification_report
y_pred = model.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_ft, y_pred))

In [0]:
# scores = model2.evaluate(ts_data, ts_pt, verbose=0)

# print("Accuracy: %.2f%%" % (scores[1]*100))
# y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)

# print(classification_report(ts_pt, y_pred))
